# Comet

![](https://user-images.githubusercontent.com/7529846/230328046-a8b18c51-12e3-4617-9b39-97614a571a2d.png)

In this guide we will demonstrate how to track your Langchain Experiments, Evaluation Metrics, and LLM Sessions with [Comet](https://www.comet.com/site/?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook).  

<a target="_blank" href="https://colab.research.google.com/github/comet-ml/comet-examples/blob/master/integrations/llm/langchain/notebooks/Comet_with_Langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Example Project:** [Comet with LangChain](https://www.comet.com/examples/comet-example-langchain-notebook/prompts?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook)

<img width="1280" alt="comet-langchain" src="https://user-images.githubusercontent.com/7529846/230326720-a9711435-9c6f-4edb-a707-94b67271ab25.png">


### Install Comet and Dependencies

In [ ]:
%pip install -U comet_llm "langchain>=0.0.346" openai google-search-results spacy textstat pandas numexpr

### Initialize Comet and Set your Credentials

You can grab your [Comet API Key here](https://www.comet.com/signup?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook) or click the link after intializing Comet

In [ ]:
import comet_llm

comet_llm.init(project="comet-example-langchain-notebook")

### Set OpenAI and SerpAPI credentials

You will need an [OpenAI API Key](https://platform.openai.com/account/api-keys) and a [SerpAPI API Key](https://serpapi.com/dashboard) to run the following examples

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "..."
# os.environ["OPENAI_ORGANIZATION"] = "..."
os.environ["SERPAPI_API_KEY"] = "..."

### Scenario 1: Using just an LLM

In [ ]:
from datetime import datetime

from langchain.callbacks.tracers.comet import CometTracer
from langchain.llms import OpenAI

comet_callback = CometTracer()
llm = OpenAI(temperature=0.9, callbacks=[comet_callback], verbose=True)

llm_result = llm.generate(["Tell me a joke", "Tell me a poem", "Tell me a fact"] * 3)
print("LLM result", llm_result)
comet_callback.flush()

### Scenario 2: Using an LLM in a Chain

In [ ]:
from langchain.callbacks.tracers.comet import CometTracer
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

comet_callback = CometTracer()
callbacks = [comet_callback]

llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(
    llm=llm, prompt=prompt_template, callbacks=callbacks, verbose=True
)

test_prompts = [{"title": "Documentary about Bigfoot in Paris"}]
print(synopsis_chain.apply(test_prompts))
comet_callback.flush()

### Scenario 3: Using An Agent with Tools 

In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain.callbacks.tracers.comet import CometTracer
from langchain.llms import OpenAI

comet_callback = CometTracer()
callbacks = [comet_callback]

llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks, verbose=True)
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    callbacks=callbacks,
    verbose=True,
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
    callbacks=callbacks,
)
comet_callback.flush()